In [1]:
import requests
import psycopg2
import pandas as pd

In [2]:
# Bước 1: Gọi API và lấy dữ liệu
api_url = "https://whiskyhunter.net/api/auctions_data/"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Failed to retrieve data: {response.status_code}")
    data = []


In [5]:
# Bước 2: Chuyển dữ liệu sang DataFrame
df = pd.DataFrame(data)

In [8]:
df.head()

,dt,winning_bid_max,winning_bid_min,winning_bid_mean,auction_trading_volume,auction_lots_count,all_auctions_lots_count,auction_name,auction_slug
0,2024-06-01,6309.7,11.5,142.13,325198.2,2288,41886,Australian Whisky Auctions,australian-whisky-auctions
1,2024-06-01,11843.2,50.8,1117.48,91633.2,82,41886,Bonhams Whisky Auction,bonhams
2,2024-06-01,10574.3,1.7,166.62,830912.6,4987,41886,Catawiki Whisky Auction,catawiki
3,2024-06-01,3045.4,4.2,269.09,27716.4,103,41886,Celtic Whiskey Auction,celtic-whiskey-auction
4,2024-06-01,2377.1,0.8,123.98,235677.9,1901,41886,Ebay Whisky Auctions,ebay-whisky-auctions


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008 entries, 0 to 2007
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   dt                       2008 non-null   object 
 1   winning_bid_max          2008 non-null   float64
 2   winning_bid_min          2008 non-null   float64
 3   winning_bid_mean         2008 non-null   float64
 4   auction_trading_volume   2008 non-null   float64
 5   auction_lots_count       2008 non-null   int64  
 6   all_auctions_lots_count  2008 non-null   int64  
 7   auction_name             2008 non-null   object 
 8   auction_slug             2008 non-null   object 
dtypes: float64(4), int64(2), object(3)
memory usage: 141.3+ KB


In [11]:
# Bước 3: Tạo cơ sở dữ liệu và kết nối với PostgreSQL
from psycopg2 import sql
# Thông tin kết nối đến PostgreSQL server
host = "localhost"
user = "postgres"
password = "1231991"
port = 5432  # Thông thường là 5432

# Kết nối đến PostgreSQL server (cơ sở dữ liệu mặc định 'postgres')
conn = psycopg2.connect(
    dbname="postgres",
    user=user,
    password=password,
    host=host,
    port=port
)
conn.autocommit = True
cur = conn.cursor()

# Tạo cơ sở dữ liệu mới
database_name = "whistky_hunter_db"
create_db_query = sql.SQL("CREATE DATABASE {}").format(
    sql.Identifier(database_name)
)

try:
    cur.execute(create_db_query)
    print(f"Cơ sở dữ liệu '{database_name}' đã được tạo thành công.")
except Exception as e:
    print(f"Lỗi khi tạo cơ sở dữ liệu: {e}")

Cơ sở dữ liệu 'whistky_hunter_db' đã được tạo thành công.


In [13]:
# Bước 4: Tạo bảng trong PostgreSQL (nếu chưa có)
create_table_query = """
CREATE TABLE IF NOT EXISTS auctions_data (
    id SERIAL PRIMARY KEY,
    dt DATE,
    winning_bid_max FLOAT,
    winning_bid_min FLOAT,
    winning_bid_mean FLOAT,
    auction_trading_volume FLOAT,
    auction_lots_count INT,
    all_auctions_lots_count INT,
    auction_name TEXT,
    auction_slug TEXT
)
"""
cur.execute(create_table_query)
conn.commit()


In [19]:
print(df.columns)

Index(['dt', 'winning_bid_max', 'winning_bid_min', 'winning_bid_mean',
       'auction_trading_volume', 'auction_lots_count',
       'all_auctions_lots_count', 'auction_name', 'auction_slug'],
      dtype='object')


In [21]:
# Bước 5: Ghi dữ liệu vào PostgreSQL
for _, row in df.iterrows():
    insert_query = """
    INSERT INTO auctions_data (dt, winning_bid_max, winning_bid_min, winning_bid_mean, auction_trading_volume, auction_lots_count, all_auctions_lots_count, auction_name, auction_slug)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_query, (
        row['dt'],
        row['winning_bid_max'],
        row['winning_bid_min'],
        row['winning_bid_mean'],
        row['auction_trading_volume'],
        row['auction_lots_count'],
        row['all_auctions_lots_count'],
        row['auction_name'],
        row['auction_slug']
    ))
    conn.commit()


In [22]:
# Đóng kết nối
cur.close()
conn.close()

print("Dữ liệu đã được ghi vào PostgreSQL thành công.")

Dữ liệu đã được ghi vào PostgreSQL thành công.
